In [1]:
import numpy as np

keyword_dict = np.load('/home/victor/catkin_ws/src/ai4hri/scripts/keyword_dict.npy',allow_pickle='TRUE').item()

keywords = keyword_dict[19]
print(keywords)

[('nikon', 0.4495), ('camera', 0.2944), ('use', 0.1597), ('yes', 0.1456), ('everyday', 0.13), ('shoot', 0.1262), ('point', 0.103), ('just', 0.0578)]


In [2]:
import mysql.connector
import os

db = mysql.connector.connect(
  host="localhost",
  user="root",
  password=os.environ.get("MYSQL_PASSWRD"),
  database="Camera_Store"
)

mycursor = db.cursor()
mycursor.execute("SELECT Model FROM Camera")

models = []
for row in mycursor:
    print(row[0])
    models.append(row[0])


Nikon Coolpix S2800
Sony Alpha a6000
Canon EOS 5D Mark III


In [3]:
from collections import Counter
import random

possible_models = []
possible_models_2 = []

for keyword in keywords:

    for row in models:

        if keyword[0] in row.lower():
            possible_models.append(row)

if len(possible_models)!=0:

    counts = Counter(possible_models)
    max_count = counts.most_common(1)[0][1]

    for value, count in counts.most_common():
        if count == max_count:
            possible_models_2.append(value)
    
    if len(possible_models_2) == 1:
        model = possible_models_2[0] 
    
    else:
        #Look the costumer and shopkeeper position to identify the camera they are refering to. For now, we choose random from the possible_models_2 list
        model = random.choice(possible_models_2)


else:
    #Look the costumer and shopkeeper position to identify the camera they are refering to. For now, we choose random from the models list
    model = random.choice(models)

# if model =! camera closest to costumer and shopkeeper position:
        #model2 = (Look the costumer and shopkeeper position to identify the camera they are using as comparison)

print(model)
#print(model2)


Nikon Coolpix S2800


In [106]:
import mysql.connector
import os

db = mysql.connector.connect(
  host="localhost",
  user="root",
  password=os.environ.get("MYSQL_PASSWRD"),
  database="Camera_Store"
)

mycursor = db.cursor()
mycursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = 'Camera_Store';")

columns = []
for row in mycursor:
    adjusted_row = row[0].replace("_", " ")
    columns.append(adjusted_row)

columns.insert(0,"this camera has a ISO up to 6400 with little noise")

print(columns)



['this camera has a ISO up to 6400 with little noise', 'Model', 'Price', 'Product ID', 'Type of camera', 'Weight', 'Automatic features', 'Product ID', 'Color', 'Product ID', 'Performance Specifications', 'Product ID', 'Employer ID', 'Product ID', 'Quantity', 'Transaction date', 'Transaction ID', 'Age', 'Email', 'Employer ID', 'First name', 'Last Name', 'Phone Number', 'Salary', 'Social Security Number', 'Store ID', 'Location', 'Manager ID', 'Store ID']


In [107]:
from sklearn.metrics.pairwise import cosine_similarity
import openai

openai.organization = os.environ.get("OPENAI_ORG_ID")
openai.api_key = os.environ.get("OPENAI_API_KEY")

model = "text-embedding-ada-002"

res = openai.Embedding.create( input = columns, engine=model)

embedded_columns=[]
for vec in res["data"]:
    embedded_columns.append(vec["embedding"])

utterance = embedded_columns[0]
embedded_columns.pop(0)
columns.pop(0)


scores = []
for i, column_candidate in enumerate(embedded_columns):
    score = cosine_similarity([utterance],[column_candidate])
    scores.append(score)

selected_columns = []
best_scores = sorted(zip(scores, columns), reverse=True)[:3]

for score in best_scores:
    selected_column= score[1].replace(" ", "_")
    selected_columns.append(selected_column)

print(selected_columns)


['Type_of_camera', 'Performance_Specifications', 'Quantity']
